In [1]:
!pip install PyPDF2 chromadb

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/21.7 MB ? eta -:--:--
    --------------------------------------- 0.5/21.7 MB 1.0 MB/s eta 0:00:21
   - -------------------------------------- 0.8/21.7 MB 1.3 MB/s eta 0:00:16
   - -------------------------------------- 1.0/21.7 MB 1.4 MB/s eta 0:00:15
   -- ------------------------------------- 1.6/21.7 MB 1.5 MB/s eta 0:00:14
   --- ------------------------------------ 1.8/21.7 MB 1.5 MB/s eta 0:00:14
   --- ------------------------------------ 2.1/21.7 MB 1.4 MB/s eta 0:00:14
   ---- ----------------------------------- 2.4/21.7 MB 1.4 MB/s eta


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from PyPDF2 import PdfReader
import chromadb
import re

In [4]:
def chunk_sentences(text, pdf_name, max_chunk_size = 500):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    current_chunk = ""
    chunks = {"text": [], "metadata": []}

    for sentence in sentences:
        # Check if adding this sentence would exceed max size
        if len(current_chunk) + len(sentence) > max_chunk_size and current_chunk:
            # Save current chunk and start new one
            chunks["text"].append(current_chunk.strip())
            chunks["metadata"].append({"Source": pdf_name,  
                                       "chunk_size": len(current_chunk.split(" ")),
                                       "character count": len(current_chunk)})
            current_chunk = sentence
        else:
            # Add sentence to current chunk
            current_chunk += " " + sentence if current_chunk else sentence
    
    # Don't forget the last chunk
    if current_chunk:
        chunks["text"].append(current_chunk.strip())
        chunks["metadata"].append({"Source": pdf_name,  
                                       "chunk_size": len(current_chunk.split(" ")),
                                       "character count": len(current_chunk)})
    
    return chunks





def retrieve_documents(pdf_path, pdf_name, max_chunk_size):
    reader = PdfReader(pdf_path)
    chunk_with_metadata = {"Documents": [], "metadata":[]}
    for page_num in range(len(reader.pages)):
        text = reader.pages[page_num].extract_text()

        #Chunking
        chunks = chunk_sentences(text, pdf_name, max_chunk_size=max_chunk_size)
        for index, document in enumerate(chunks["text"]):
            chunk_with_metadata["Documents"].append(document)
            chunk_with_metadata["metadata"].append(chunks["metadata"][index])
    return chunk_with_metadata
        





In [5]:
retrieve_documents("Python For Dummies.pdf", "Python for Dummies", 400)

{'Documents': ['Beginning \nProgramming \nwith Python®',
  'Beginning \nProgramming \nwith Python®\nby John Paul Mueller',
  'Beginning Programming with Python® For Dummies®\nPublished by: John Wiley & Sons, Inc., 111 River Street, Hoboken, NJ 07030-5774, www.wiley.com\nCopyright © 2014 by John Wiley & Sons, Inc., Hoboken, New Jersey\nMedia and software compilation copyright © 2014 by John Wiley & Sons, Inc. All rights reserved.',
  'Published simultaneously in Canada\nNo part of this publication may be reproduced, stored in a retrieval system or transmitted in any form or \nby any means, electronic, mechanical, photocopying, recording, scanning or otherwise, except as permit-\nted under Sections 107 or 108 of the 1976 United States Copyright Act, without the prior written permis -\nsion of the Publisher.',
  'Requests to the Publisher for permission should be addressed to the Permissions \nDepartment, John Wiley & Sons, Inc., 111 River Street, Hoboken, NJ 07030, (201) 748-6011, fax (2